In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2022-02-06 09:43:28--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-02-06 09:43:28--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-02-06 09:43:29--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [2]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [3]:
import os

In [4]:
EMBEDDING_DIMENSION=50

In [13]:
glove_weights_file_path = os.path.join(f'glove.6B.{EMBEDDING_DIMENSION}d.txt')

In [6]:
glove_weights_file_path

'glove.6B.50d.txt'

In [8]:
import numpy as np

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
word_index=None
file = open(glove_weights_file_path)
tokenizer = Tokenizer(num_words=40000, lower= 1, oov_token="<OOV>")
tokenizer.fit_on_texts(file)
word_index = tokenizer.word_index

In [29]:
weights=[]
file = open(glove_weights_file_path)
for index, line in enumerate(file):
    values = line.split() # Word and weights separated by space
    word = values[0] # Word is first symbol on each line
    #print(word)
    word_weights = np.asarray(values[1:], dtype=np.float32) # Remainder of line is weights for word
    #print(word_weights)
    word_index[word] = index + 1 # PAD is our zeroth index so shift by one
    #print(word_index[word])
    weights.append(word_weights)

    if index + 1 == 40_000:
        # Limit vocabulary to top 40k terms
        print('break')
        break



break


In [31]:
len(weights[0])

50

In [32]:
EMBEDDING_DIMENSION = len(weights[0])
# Insert the PAD weights at index 0 now we know the embedding dimension
weights.insert(0, np.random.randn(EMBEDDING_DIMENSION))

# Append unknown and pad to end of vocab and initialize as random
UNKNOWN_TOKEN=len(weights)
word_index['UNK'] = UNKNOWN_TOKEN
weights.append(np.random.randn(EMBEDDING_DIMENSION))

# Construct our final vocab
weights = np.asarray(weights, dtype=np.float32)

VOCAB_SIZE=weights.shape[0]

In [33]:
import nltk

In [35]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [36]:
features = {}
features['word_indices'] = nltk.word_tokenize('hello world') # ['hello', 'world']
features['word_indices'] = [word_index.get(word, UNKNOWN_TOKEN) for word in features['word_indices']]

In [44]:
VOCAB_LENGTH=40002 # +2 unknown tokens

In [45]:
glove_weights_initializer = tf.constant_initializer(weights)
embedding_weights = tf.compat.v1.get_variable(
    name='embedding_weights',
    shape=(VOCAB_LENGTH, EMBEDDING_DIMENSION),
    initializer=glove_weights_initializer,
    trainable=False)
embedding = tf.nn.embedding_lookup(embedding_weights, features['word_indices'])